<a href="https://colab.research.google.com/github/JMindpalace/Machine_Learning/blob/main/3.0%20%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

< 해당 페이지는 **특성(Feature)**을 파악하고, **특성 공학(Feature Engineering)**으로 증폭 혹은 축소 시키는 페이지입니다. >

> 데이터 변경은 거의 없고, 모델링의 전처리로 특성공학이나 상관계수 등에 따라 줄이는 것이 목표입니다.

> 기대 결과 값: 특성 간의 상관관계를 파악하고, 적절한 데이터를 분리합니다

# 데이터 분석(Data Analysis)

## 데이터 분석의 방향성

- 지도학습적 목적 : 특성과 타겟의 관계를 파악

- 비지도학습적 목적 : 특성간의 관계를 파악해서 예측 타겟 설정



---



---



### 데이터셋 분석

In [ ]:
# 특정 조건 검색1
df[ (df['col'] == 'condition') ] # []내에 ()와 &로 다중 조건 검색 가능

# 특정 조건 검색2 - 쿼리문의 변수는 @로 가져와야함(@변수)
df.query( ('col' == 'condition') ) # ()내에 ()와 and로 다중 조건 검색 가능

In [ ]:
# Groupby - mean()외에도 median() 등 사용 가능
df.groupby('그룹 기준 칼럼').mean()['값을 볼 칼럼'] # [기준1, 기준2]로 대/중분류, [ [값1, 값2] ]로 다중 그룹
# Groupby가 리스트 반환이지만, 기준 칼럼 뒤에 , as_index=False면 데이터 프레임으로 반환

In [ ]:
# feature 상관관계
numeric_feature = df.dtype[ df.dtypes != 'object' ].index
df[numeric_feature].corr()['target'].sort_values(ascending=False).head(5)



---



---



### 데이터셋 분리: 모델 일반화 검증을 위한 분리
> 기본 [훈련+테스트] + 추가 검증셋(모델 평가와 개선에 사용)

> 단, 테스트셋은 학습셋과 동일 특징이 필수이며<br>
> 테스트셋에 적합한 모델 방지를 위해 1회 사용을 원칙으로함

In [ ]:
from sklearn.model_selection import train_test_split # Hold-Out Validation
from sklearn.model_selection import KFold            # CV - KFold
from sklearn.model_selection import cross_val_score  # CV - Score

In [ ]:
# 만약 데이터셋 분리 시 copy 사용 고려, 단 object로 복사되기에 수치형에서는 추가적 변환 필요
af = df.copy()
train, test = af.sample(frac=0.75), af.drop(train.index)

#### n-way Hold-Out Validation
> 데이터 크기가 작다면 부적합(충분한 학습 불가능)<br>
> 데이터 분리를 랜덤으로 진행하여 모델 신뢰성에 의문 제기

In [ ]:
# 2-way: 학습(Train)과 테스트(Test)
train2 , test = train_test_split( af, test_size=0.2 )

# 3-way: 2way + 검증(Validation)
train, val = train_test_split( train2, train_size=0.8 )

# 타겟과 특성 분리 - 함수 사용 전 target과 feature를 지정해야함
def X_y_split(df):
  X = df[features] # df.drop('target', axis=1)
  y = df[target]
  return X, y

X_train, y_train = X_y_split(train) # val, test도 같은 방식으로 분리

In [ ]:
# 데이터 분리 후 확인
af.shape == train.index.count() + test.index.count()
train.shape == X_train.index.count() + y_train.index.count()

In [ ]:
# 데이터 범위 조정
X_train = X_train.reshape(-1, 1)
X_test = X_test.reshape(-1, 1)

In [ ]:
# 데이터 분리 검증 - 5.0 모델학습의 평가부분 참고

#### Cross Validation(교차검증): 훈련 + 테스트로 진행
> 모든 샘플이 검증에 사용(k번 학습과 검증 반복)되지만<br>
> 테스트셋 분리도 필수임

In [ ]:
# k-ford CV: k개로 분할 후 예측성능결과를 평균으로 도출
X_train2, y_train2 = X_y_split(train2)

kf = KFold(n_splits=n) # kf.get_n_splits()로 분할 갯수 k를 알 수 있음
cv_result = []

for train_idx, test_idx in kf.split(X_train2) : # 반복마다 서로다른 k-1개는 훈련, 1개는 검증으로 분리
  X_train_cv, X_val_cv = X_train2.iloc[train_idx], X_train2.iloc[test_idx]
  y_train_cv, y_val_cv = y_train2.iloc[train_idx], y_train2.iloc[test_idx]

  model.fit(X_train_cv, y_train_cv) # 모델 학습
  pred_cv = model.predict(X_val_cv) # 모델 예측
  mae_cv = mean_absolute_error(y_val_cv, pred_cv).round(2) # 검증결과 종합으로 MAE 평가
  cv_result.append(mae_cv)

# cv_result는 MAE 리스트, np.mean(cv_result)과 np.std(cv_result)

cv_results = cross_val_score( # cv_results 호출 시 MAE로 평가된 리스트 반환
    model,
    X_train_cv,
    y_train_cv,
    cv = n,     # kf
    scoring = 'neg_mean_absolute_error'
    n_jobs = -1
)